# DunedinPACE

## Index
1. [Instantiate model class](#Instantiate-model-class)
2. [Define clock metadata](#Define-clock-metadata)
3. [Download clock dependencies](#Download-clock-dependencies)
5. [Load features](#Load-features)
6. [Load weights into base model](#Load-weights-into-base-model)
7. [Load reference values](#Load-reference-values)
8. [Load preprocess and postprocess objects](#Load-preprocess-and-postprocess-objects)
10. [Check all clock parameters](#Check-all-clock-parameters)
10. [Basic test](#Basic-test)
11. [Save torch model](#Save-torch-model)
12. [Clear directory](#Clear-directory)

Let's first import some packages:

In [1]:
import os
import inspect
import shutil
import json
import torch
import pandas as pd
import pyaging as pya

## Instantiate model class

In [2]:
def print_entire_class(cls):
    source = inspect.getsource(cls)
    print(source)

print_entire_class(pya.models.DunedinPACE)

class DunedinPACE(pyagingModel):
    def __init__(self):
        super().__init__()

    def preprocess(self, x):
        """
        Apply quantile normalization on x using gold standard means.
        """
        # Ensure gold_standard_means is a 1D tensor and sorted
        sorted_gold_standard = torch.sort(
            torch.tensor(self.reference_values, device=x.device, dtype=x.dtype)
        )[0]

        # Pre-compute the quantile indices
        quantile_indices = torch.linspace(
            0, len(sorted_gold_standard) - 1, steps=x.size(1)
        ).long()

        # Prepare a tensor to hold normalized data
        normalized_data = torch.empty_like(x, device=x.device, dtype=x.dtype)

        for i in range(x.size(0)):
            sorted_indices = torch.argsort(x[i, :])
            normalized_data[i, sorted_indices] = sorted_gold_standard[quantile_indices]

        # Return only the subset from x that is used in the base model
        return normalized_data[:, self.preprocess_

In [3]:
model = pya.models.DunedinPACE()

## Define clock metadata

In [4]:
model.metadata["clock_name"] = 'dunedinpace'
model.metadata["data_type"] = 'methylation'
model.metadata["species"] = 'Homo sapiens'
model.metadata["year"] = 2022
model.metadata["approved_by_author"] = '✅'
model.metadata["citation"] = "Belsky, Daniel W., et al. \"DunedinPACE, a DNA methylation biomarker of the pace of aging.\" Elife 11 (2022): e73420."
model.metadata["doi"] = "https://doi.org/10.7554/eLife.73420"
model.metadata["research_only"] = True
model.metadata["notes"] = "This model is for research purposes only. Commercial users should contact exclusive DunedinPACE licensee TruDiagnosticTM. The automatic failure if fewer than 80% of the CpG probes are available is not implemented and left to the user's discretion."

## Download clock dependencies

#### Download from R package

In [5]:
%%writefile download.r

options(repos = c(CRAN = "https://cloud.r-project.org/"))
library(jsonlite)

install.packages("devtools")
devtools::install_github("danbelsky/DunedinPACE", build_vignettes = FALSE)
library(DunedinPACE)

PACE_list = list(c(
    mPACE_Models$model_names, 
    mPACE_Models$gold_standard_probes, 
    mPACE_Models$model_weights, 
    mPACE_Models$model_intercept,
    mPACE_Models$model_means,
    mPACE_Models$model_probes,
    mPACE_Models$gold_standard_means    
))

write_json(PACE_list, "DunedinPACE.json", digits = 12)

Writing download.r


In [6]:
os.system("Rscript download.r")

0

## Load features

#### From JSON file

In [7]:
with open('DunedinPACE.json', 'r') as f:
    PACE_list = json.load(f)[0]
model.features = PACE_list['DunedinPACE']
model.base_model_features = PACE_list['DunedinPACE.4']

## Load weights into base model

In [8]:
weights = torch.tensor(PACE_list['DunedinPACE.1']).unsqueeze(0).float()
intercept = torch.tensor([PACE_list['DunedinPACE.2'][0]]).float()

#### Linear model

In [9]:
base_model = pya.models.LinearModel(input_dim=len(model.features))

base_model.linear.weight.data = weights.float()
base_model.linear.bias.data = intercept.float()

model.base_model = base_model

## Load reference values

In [10]:
model.reference_values = PACE_list['DunedinPACE.5']

## Load preprocess and postprocess objects

In [11]:
model.preprocess_name = 'quantile_normalization_with_gold_standard'

indices = [model.features.index(item) for item in model.base_model_features]
model.preprocess_dependencies = [indices]

In [12]:
model.postprocess_name = None
model.postprocess_dependencies = None

## Check all clock parameters

In [13]:
pya.utils.print_model_details(model)


%==================================== Model Details ====================================%
Model Attributes:

training: True
metadata: {'approved_by_author': '✅',
 'citation': 'Belsky, Daniel W., et al. "DunedinPACE, a DNA methylation '
             'biomarker of the pace of aging." Elife 11 (2022): e73420.',
 'clock_name': 'dunedinpace',
 'data_type': 'methylation',
 'doi': 'https://doi.org/10.7554/eLife.73420',
 'notes': 'This model is for research purposes only. Commercial users should '
          'contact exclusive DunedinPACE licensee TruDiagnosticTM. The '
          'automatic failure if fewer than 80% of the CpG probes are available '
          "is not implemented and left to the user's discretion.",
 'research_only': True,
 'species': 'Homo sapiens',
 'version': None,
 'year': 2022}
reference_values: [0.8499678448731, 0.7897610893879, 0.9717597609504, 0.8480138104804, 0.2475593905826, 0.1097207575569, 0.189885225724, 0.3909313843346, 0.9168688056148, 0.2708397202163, 0.87366963

## Basic test

In [14]:
torch.manual_seed(42)
input = torch.randn(10, len(model.features), dtype=float)
model.eval()
model.to(float)
pred = model(input)
pred

tensor([[ 0.2938],
        [ 0.8014],
        [-0.0384],
        [-0.2034],
        [-0.1848],
        [ 0.5177],
        [-0.1668],
        [ 0.3322],
        [-0.8489],
        [ 0.7232]], dtype=torch.float64, grad_fn=<AddmmBackward0>)

## Save torch model

In [15]:
torch.save(model, f"../weights/{model.metadata['clock_name']}.pt")

## Clear directory
<a id="10"></a>

In [16]:
# Function to remove a folder and all its contents
def remove_folder(path):
    try:
        shutil.rmtree(path)
        print(f"Deleted folder: {path}")
    except Exception as e:
        print(f"Error deleting folder {path}: {e}")

# Get a list of all files and folders in the current directory
all_items = os.listdir('.')

# Loop through the items
for item in all_items:
    # Check if it's a file and does not end with .ipynb
    if os.path.isfile(item) and not item.endswith('.ipynb'):
        os.remove(item)
        print(f"Deleted file: {item}")
    # Check if it's a folder
    elif os.path.isdir(item):
        remove_folder(item)

Deleted file: download.r
Deleted file: DunedinPACE.json
